### Init libraries and load data

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 29 06:43:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import time
import os, shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, UnitNormalization, RandomFlip, RandomRotation, RandomBrightness, RandomContrast
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2, VGG19, MobileNetV2, ResNet152V2, ResNet50
from skimage.transform import resize
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


from google.colab import drive
drive.mount("/content/gdrive")

path = '/content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition'
final_pass_dir = path+'/FINAL PASSES'
#augmented_dir = path+'/VICTOR_SEPT2023/saved_augmented_models'
results_dir =  path+'/DATA_ANALYSIS_FINALS/Results_Turtles'
npy_file=path + '/DATA_HUMBOLDT_FINAL/turtles_crop_300_300_100_color.npy'

data=np.load(npy_file, allow_pickle=True)
images=data['np_images']
if np.max(images) > 1: images = images/255.0

df = data['info_images']





Mounted at /content/gdrive


In [3]:
df.head()

pic                                        path  ind_name  \
0  10230413_0000.JPG  ./Tortugas_crop/10230413/10230413_0000.JPG  10230413   
1  10230413_0003.JPG  ./Tortugas_crop/10230413/10230413_0003.JPG  10230413   
2  10230413_0004.JPG  ./Tortugas_crop/10230413/10230413_0004.JPG  10230413   
3  10230413_0008.JPG  ./Tortugas_crop/10230413/10230413_0008.JPG  10230413   
4  10230413_0013.JPG  ./Tortugas_crop/10230413/10230413_0013.JPG  10230413   

  group    P  
0     A   50  
1     A   80  
2     A  100  
3     A   40  
4     A   80

In [4]:
images.shape

(2200, 300, 300, 3)

The following function must be used if you are using Keras 3, but loading models saved in Keras 2.15


In [8]:
def my_load_model(path, compile=None):
    """Loads a TensorFlow SavedModel and wraps it in a Keras Model for inference.

    Args:
      path: The path to the TensorFlow SavedModel directory.

    Returns:
      A Keras Model that can be used for inference with predict_proba.
    """


    # Load the SavedModel as a TFSMLayer
    tfsm_layer = tf.keras.layers.TFSMLayer(
        path, call_endpoint='serving_default'  # Adjust call_endpoint if necessary
    )

    '''
    # We know the input size, so no need for all this which slow down the code
    # extract the input shape
    loaded_model = tf.saved_model.load(path)

    # Access the specific endpoint (e.g., 'serving_default')
    concrete_func = loaded_model.signatures['serving_default']

    # Get the input shapes from the concrete function
    for input_name, tensor_spec in concrete_func.structured_input_signature[1].items():
        input_shape = tensor_spec.shape[1:]
    '''

    input_shape = (300,300,3)

    # Create a Keras Input layer with the required shape
    input_layer = tf.keras.Input(shape=input_shape)



    # Pass the input layer to the TFSMLayer
    output_tensor = tfsm_layer(input_layer)

    # Create a Keras Model
    model = tf.keras.Model(inputs=input_layer, outputs=output_tensor)


    model.model_path = path

    return model


def get_input_shape(model):

    model_path = model.model_path
    ## we add the input shape to the model
    loaded_model = tf.saved_model.load(model_path)

    # Access the specific endpoint (e.g., 'serving_default')
    concrete_func = loaded_model.signatures['serving_default']

    # Get the input shapes from the concrete function
    for input_name, tensor_spec in concrete_func.structured_input_signature[1].items():
        input_shape = tensor_spec.shape


    return input_shape


if tf.keras.__version__[0]=='3':
  print('Substituing keras load_model by custom version')
  load_model = my_load_model

Substituing keras load_model by custom version


### Define Functions

In [13]:
def create_translation_dict(string_list):
    unique_strings = sorted(set(string_list))
    translation_dict = {string: i for i, string in enumerate(unique_strings)}
    return translation_dict

def translate_to_integers(string_list, translation_dict):
    return [translation_dict[string] for string in string_list]

def entropy(p, normalize=True ):
    # entropy of single vector
    p_n=p/np.sum(p) #normalize just in case
    logs = np.array([0 if x==0 else np.log(x) for x in p_n])
    S = - np.sum(p_n*logs)
    if normalize:
        S=S/np.log(len(p))
    return S

def proba2Entropy(y_proba):
    #entropy of dataset of vectors
    L = len(y_proba)
    S = np.zeros(L)
    for i in range(L):
      S[i] = entropy(y_proba[i])
    return S

def predict_proba(X_data, model):
    '''try:
      n_channels = model.layers[0].input_shape[0][-1]
    except:
      n_channels = get_input_shape(model)[-1]
    '''
    n_channels = 3 # we know this for sure for all turtle models

    if n_channels ==1:
        X_t = np.squeeze(tf.image.rgb_to_grayscale(X_data).numpy())
    else:
        X_t = X_data

    y_proba_pred = model.predict(X_t)['classification_layer']
    return y_proba_pred


def predict(X_data, model):
    y_proba_pred = predict_proba(X_data, model)
    class_pred = y_proba_pred.argmax(axis=1)
    return class_pred


def  predict_proba_ensemble(X_data, ensemble, detail_prediction = False):
    predictions_list = []
    for model in ensemble:
        y_proba_pred = predict_proba(X_data, model)
        predictions_list.append(y_proba_pred)

    all_preds = np.stack(predictions_list, axis=-1)
    av_pred = np.mean(all_preds, axis=-1)
    av_pred = av_pred[:, :, np.newaxis]
    if detail_prediction:
      out = np.concatenate((av_pred, all_preds), axis=-1)
    else: out = av_pred

    return out

def  predict_ensemble(X_data, ensemble, detail_prediction = False):
    full_prediction_proba = predict_proba_ensemble(X_data, ensemble, detail_prediction = detail_prediction)
    class_pred = full_prediction_proba.argmax(axis=1)

    return class_pred


def evaluate_metrics(test_pred, y_test):
    true_labels = y_test
    N_models = test_pred.shape[-1]

    model_metrics = {}

    for i in range(N_models):
        predictions = test_pred[:,i]
        accuracy = accuracy_score(true_labels, predictions)
        recall = recall_score(true_labels, predictions, average='macro')
        precision = precision_score(true_labels, predictions, average='macro')
        f1 = f1_score(true_labels, predictions, average='macro')

        namemodel = f'model{i}' if i>0 else 'ensemble'

        model_metrics[namemodel] = {
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'f1': f1
        }

    metrics_df = pd.DataFrame(model_metrics)
    return metrics_df

#### DATA GENERATIONS ###

def get_data_A():

  A_test_select = df['P']==100
  A_test = images[A_test_select]
  A_test_classes = df[df['P']==100]['ind_name'].values

  class_dict_test = create_translation_dict(A_test_classes)
  A_y_test = translate_to_integers(A_test_classes, class_dict_test)

  # Generic names for all exps.
  X_test = A_test.copy()
  y_test = A_y_test.copy()

  return X_test, y_test




def create_ensemble_from_list(model_paths, verbatim = True, fullpath=False):
    N_models = len(model_paths)
    ensemble = []
    print('Loading models...')
    for i,model_name in enumerate(model_paths):
      if not fullpath:
        full_model_name = final_pass_dir+'/'+model_name+'/model'
      else:
        full_model_name = model_name+'/model'


        NN = load_model(full_model_name, compile = False)
        ensemble.append(NN)
        if verbatim:
          print(f'model {i+1} of {N_models} added: {model_name}')
    return ensemble


def create_ensemble(MODEL_TO_USE = 'T_FINAL', verbatim = True):
    model_paths = [mp for mp in os.listdir(final_pass_dir) if MODEL_TO_USE in mp]
    N_models = len(model_paths)
    print(model_paths)
    if N_models==0:
        print(f'== NO MODELS WITH THAT CODE ==')
    else:
        CL = ('CL' in model_paths[0])
        print(f'There are {N_models} models, and CL mode is {CL}')


    #if not CL: X_test = np.squeeze(tf.image.rgb_to_grayscale(X_test).numpy())

    ensemble = []
    print('Loading models...')
    for i,model_name in enumerate(model_paths):
        full_model_name = final_pass_dir+'/'+model_name+'/model'

        NN = load_model(full_model_name, compile = False)
        ensemble.append(NN)
        if verbatim:
          print(f'model {i+1} of {N_models} added: {model_name}')
    return ensemble





## Select EXP and test sets

In [14]:
X_test_A, y_test_A  = get_data_A()



## Experiment A.

**Exploration of models with P=60  + final pass**

Here, we have the final network with some modifications (batch_size in [2,4,8], and gray in [False, True]

For each of the **versions**,  we have computed 5 instances, and we report different performance metrics (accuracy, recall, precission, F1-score) for each each instance. we also report the average of the instance metrics, and the metrics for the ensemble model.

The 60+ models availables are the following
- H_A_09  (batch = 4, gray = False)
- H_A_10  (batch = 4, gray = True)
- H_A_11  (batch = 2, gray = False)
- H_A_12  (batch = 2, gray = True)
- H_A_13  (batch = 8, gray = False)
- H_A_14  (batch = 8, gray = True)

All this models are in the Final Passes directory, and have at least 5 instances.




## Create and Evaluate Ensemble for KNOWNS.

These models are all p=60, and a final pass with the validation test included (val =20%, so it is a p=80% effective model) and for each model we have ensemble of 5.
Final accuracy of the ensemble models are saved in csv files



In [15]:
recalculate=True
models_A = ['T_A_01_FINAL']
all_models_A = [final_pass_dir+'/'+m for m in os.listdir(final_pass_dir) if m[0:3]=='T_A']
print(all_models_A)

name_out = 'ensemble_turtle_Final_Passes'
metrics_A = []

if recalculate:
  for this_model in models_A:
    print(this_model)
    p=60
    models_to_aggregate = [m for m in all_models_A if this_model in m][0:5]
    ensemble = create_ensemble_from_list(model_paths = models_to_aggregate, verbatim = True, fullpath=True)
    A_test_pred_proba = predict_proba_ensemble(X_test_A, ensemble, detail_prediction = True)
    A_test_pred = predict_ensemble(X_test_A, ensemble, detail_prediction = True)
    df_metric = evaluate_metrics(A_test_pred, y_test_A)
    print(f'saving model {this_model}')
    df_metric['model'] = this_model
    df_metric['average'] =  df_metric[['model1', 'model2', 'model3', 'model4', 'model5']].mean(axis=1)
    df_metric['std'] =  df_metric[['model1', 'model2', 'model3', 'model4', 'model5']].std(axis=1)
    df_metric['P'] = p


    # Retain only 5 models
    df_metric= df_metric[['model','P', 'ensemble',	'average', 'std', 'model1',	'model2',	'model3',	'model4',	'model5']]
    name_to_save = results_dir+'/'+name_out+'_'+this_model+'.csv'
    #df_metric.to_csv(results_dir+'/'+name_out+'_'+this_model+'.csv', sep=',', index=True)
    metrics_A.append(df_metric)

  metric_df_A = pd.concat(metrics_A)
  metric_df_A.reset_index(inplace=True, drop=False)
  metric_df_A.rename(columns = {'index':'metric'}, inplace=True)
  metric_df_A.to_csv(results_dir+'/'+name_out+'.csv', sep=',', index=False)

  metric_df_A


['/content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/FINAL PASSES/T_A_01_FINAL-FINAL_PASS_01', '/content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/FINAL PASSES/T_A_01_FINAL-FINAL_PASS_02', '/content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/FINAL PASSES/T_A_01_FINAL-FINAL_PASS_03', '/content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/FINAL PASSES/T_A_01_FINAL-FINAL_PASS_04', '/content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/FINAL PASSES/T_A_01_FINAL-FINAL_PASS_05']
T_A_01_FINAL
Loading models...
model 1 of 5 added: /content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/FINAL PASSES/T_A_01_FINAL-FINAL_PASS_01
model 2 of 5 added: /content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/FINAL PASSES/T_A_01_FINAL-FINAL_PASS_02
model 3 of 5 added: /content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/FINAL PASSES/T_A_01_FINAL-FINAL_PASS_03
model 4 of 5 added: /content/gdrive/MyDrive/PenguinRe

In [16]:
results_dir+'/'+name_out+'_'+this_model+'.csv'

'/content/gdrive/MyDrive/PenguinRecogMikeVictor/chest_recognition/DATA_ANALYSIS_FINALS/Results_Turtles/ensemble_turtle_Final_Passes_T_A_01_FINAL.csv'

We read the results, specially the **accuracy per model**




In [17]:
name_out = 'ensemble_turtle_Final_Passes'
metric_df_A = pd.read_csv(results_dir+'/'+name_out+'.csv', sep=',')
accuracy_A = metric_df_A[metric_df_A['metric']=='accuracy']
accuracy_A

metric         model   P  ensemble   average       std    model1  \
0  accuracy  T_A_01_FINAL  60  0.997727  0.988636  0.011918  0.995455   

     model2    model3    model4    model5  
0  0.968182  0.997727  0.993182  0.988636